## Network Data Processing for Graph Construciton

After discussion with the Network team, we defined a graph schema based on **Metro** and **Port** only. There are two types of nodes and four types of edges as listed below.

- Node types:
    - Metro: all metros in network;
    - Port: all connected ports

- Edge types:
    - (Metro, demand, Metro): a directional link from a Metro to any other Metros. This is a fully connected sub-graph, i.e., any two metros have two links.
    - (Port, in, Metro): a bi-directional link from a Port to a Metro. There will be a reverse link from a Metro to a Port.
    - (Metro, to, Metro): a directional link from a Port to a Port. Following the physical connection, there is only one way Port-to-Port traffic.
----

In [1]:
import os
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', 100)

In [2]:
nl04_df = pd.read_csv('node_link_data_2023121104.csv')
nl05_df = pd.read_csv('node_link_data_2023121105.csv')
print(nl04_df.shape)
print(nl05_df.shape)

split_line = '-'.join([_ for i in range(100)])
print(split_line)

dm04_df = pd.read_csv('traffic_demand_2023121104.csv')
dm05_df = pd.read_csv('traffic_demand_2023121105.csv')
print(dm04_df.shape)
print(dm05_df.shape)

(383412, 22)
(383362, 22)
---------------------------------------------------------------------------------------------------
(111834, 4)
(111625, 4)


In [3]:
nl04_df[nl04_df.traffic_gbps.isna()].sample(1)

,site_a,site_z,device_a,device_z,port_a,port_z,path_id,bw_gbps,lifecycle_state,tshift_status_a,tshift_status_z,metro_a,metro_z,span_name_m,span_name,fabric,span_type,span_build_type,span_bw_gbps,timestamp,traffic_gbps,utilization_%
158807,MDW062,SEA019,mdw62-br-fnc-f1-b1-t1-r14,sea19-br-fnc-f1-b1-t1-r14,jrp10-3,jrp10-3,P-MDW062-SEA019-01-[BF]-Lumen,10.0,PENDING_NORMALIZE,shifted,shifted,MDW,SEA,MDW_SEA,MDW062_SEA019,FNC,BACKBONE,BACKBONE,1080.0,2023-12-11 04:24:00,NaN,NaN


In [4]:
nl05_df[nl05_df.traffic_gbps.isna()].head(1)

,site_a,site_z,device_a,device_z,port_a,port_z,path_id,bw_gbps,lifecycle_state,tshift_status_a,tshift_status_z,metro_a,metro_z,span_name_m,span_name,fabric,span_type,span_build_type,span_bw_gbps,timestamp,traffic_gbps,utilization_%
5,PDX001,PDX068,pdx1-br-fnc-f1-b3-t1-r2,pdx68-br-fnc-f1-b1-t1-r14,jrp7-4,jrp11-4,P-PDX001-PDX068-02-[BF],100.0,DECOMMISSIONED,shifted,shifted,PDX,PDX,PDX_PDX,PDX001_PDX068,FNC,INTRA-METRO,BULK_FIBER,153600.0,2023-12-11 05:04:00,NaN,NaN


### Step 1: Collect all nodes

- Collect all Metro from both traffic and demand tables, and get unique values;
- Collect all device+port name from traffic tables, and get unique values.

In [5]:
nl_df = pd.concat([nl04_df, nl05_df])
dm_df = pd.concat([dm04_df, dm05_df])

In [6]:
print(nl_df.columns)
print(dm_df.columns)

Index(['site_a', 'site_z', 'device_a', 'device_z', 'port_a', 'port_z',
       'path_id', 'bw_gbps', 'lifecycle_state', 'tshift_status_a',
       'tshift_status_z', 'metro_a', 'metro_z', 'span_name_m', 'span_name',
       'fabric', 'span_type', 'span_build_type', 'span_bw_gbps', 'timestamp',
       'traffic_gbps', 'utilization_%'],
      dtype='object')
Index(['src_region', 'dst_region', 'timebucket', 'gb'], dtype='object')


In [7]:
# fill NaNs with 0s
print(nl_df[nl_df.traffic_gbps.isna()].shape)
nl_df.fillna(0, inplace=True)
print(nl_df[nl_df.traffic_gbps.isna()].shape)

(38501, 22)
(0, 22)


In [8]:
# drop duplicated records
print(nl_df.shape)
nl_df.drop_duplicates(inplace=True)
print(nl_df.shape)

(766774, 22)
(766774, 22)


#### **Metro** nodes

In [9]:
metro_a = nl_df.metro_a.unique()
metro_z = nl_df.metro_a.unique()
print(metro_a.shape)
print(metro_z.shape)
metro_in_nl = np.char.lower(np.unique(np.concatenate([metro_a, metro_z])).astype('U'))
print(metro_in_nl.shape)
metro_in_nl[:7]

(93,)
(93,)
(93,)


array(['akl', 'ams', 'arn', 'ath', 'atl', 'bah', 'bcn'], dtype='<U3')

In [10]:
src = dm_df.src_region.unique()
dst = dm_df.dst_region.unique()
print(src.shape)
print(dst.shape)
metro_in_dm = np.unique(np.concatenate([src, dst])).astype('U')
print(metro_in_dm.shape)
metro_in_dm[:7]

(114,)
(132,)
(132,)


array(['abc', 'akl', 'amm', 'ams', 'arn', 'ath', 'atl'], dtype='<U3')

In [11]:
metro_all = np.unique(np.concatenate([metro_in_nl, metro_in_dm]))
print(metro_all.shape)
metro_all[-7:]

(133,)


array(['yul', 'yvr', 'yyc', 'zag', 'zaz', 'zhy', 'zrh'], dtype='<U3')

#### **Port** nodes

In [12]:
nl_df['da_pa'] = nl_df['device_a'] + ':' + nl_df['port_a']
nl_df['dz_pz'] = nl_df['device_z'] + ':' + nl_df['port_z']
da_pa_df = nl_df.da_pa.unique()
dz_pz_df = nl_df.dz_pz.unique()
print(da_pa_df.shape)
print(dz_pz_df.shape)
port_all = np.unique(np.concatenate([da_pa_df, dz_pz_df]))
print(port_all.shape)
port_all[:7]

(31291,)
(31126,)
(62334,)


array(['akl50-br-cor-r1:et-0/0/19', 'akl50-br-cor-r2:et-0/0/19',
       'akl50-br-cor-r3:et-0/0/19', 'ams1-br-bfb-f1-b2-t1-r1:et-0/0/13',
       'ams1-br-bfb-f1-b2-t1-r1:et-0/0/17',
       'ams1-br-bfb-f1-b2-t1-r1:et-0/0/23',
       'ams1-br-bfb-f1-b2-t1-r1:et-0/0/29'], dtype=object)

### Step 2: Generate node data

collect all nodes' feature. For **Metro** nodes, we can have the number of sites and the number of devices as features. And if possible to get all longitute and latitute as feature too. For **Port** nodes, there is no feature, but a node id list.

In [13]:
metro_df = pd.DataFrame({'nid': metro_all})
metro_df.shape

(133, 1)

In [14]:
nl_df.head(1)

,site_a,site_z,device_a,device_z,port_a,port_z,path_id,bw_gbps,lifecycle_state,tshift_status_a,tshift_status_z,metro_a,metro_z,span_name_m,span_name,fabric,span_type,span_build_type,span_bw_gbps,timestamp,traffic_gbps,utilization_%,da_pa,dz_pz
0,IAD055,IAD061,iad55-br-fnc-f1-b2-t1-r9,iad61-br-fnc-f1-b1-t1-r1,jrp10-4,jrp14-4,P-IAD055-IAD061-01-[Metro],100.0,OPERATIONAL,not_shifted,not_shifted,IAD,IAD,IAD_IAD,IAD055_IAD061,FNC,INTRA-METRO,AMAZON_OLP_METRO,76800.0,2023-12-11 04:04:00,8.162865,8.162865,iad55-br-fnc-f1-b2-t1-r9:jrp10-4,iad61-br-fnc-f1-b1-t1-r1:jrp14-4


In [15]:
def split_site(site):
    return [site[:3], site[3:]]

In [16]:
site_a = pd.DataFrame(nl_df.site_a.apply(split_site).to_list(), columns=['metro', 'site_num'])
site_z = pd.DataFrame(nl_df.site_z.apply(split_site).to_list(), columns=['metro', 'site_num'])

In [17]:
sites = pd.concat([site_a, site_z])

In [18]:
print(sites.shape)
sites.head()

(1533548, 2)


,metro,site_num
0,IAD,055
1,LAX,003
2,CMH,051
3,FRA,053
4,BER,028


In [19]:
uniq_sites = sites.drop_duplicates()
print(uniq_sites.shape)
uniq_sites.sample(4)

(368, 2)


,metro,site_num
162,BOG,051
17,IAD,117
164,LHR,005
122,PIT,050


In [20]:
site_cnt = pd.DataFrame(uniq_sites.groupby('metro').count())

In [21]:
site_cnt = site_cnt.reset_index()

In [22]:
site_cnt['metro_id'] = site_cnt.metro.str.lower()
print(site_cnt.shape)
site_cnt.head()

(108, 3)


,metro,site_num,metro_id
0,AKL,1,akl
1,AMS,5,ams
2,ARN,4,arn
3,ATH,1,ath
4,ATL,6,atl


In [23]:
metro_node = pd.merge(metro_df, site_cnt, how='left', left_on='nid', right_on='metro_id')

In [24]:
print(metro_node.shape)

(133, 4)


In [25]:
metro_node.sample(7)

,nid,metro,site_num,metro_id
29,cph,CPH,1.0,cph
43,for,FOR,1.0,for
91,osu,NaN,NaN,NaN
8,bcn,BCN,1.0,bcn
73,lux,LUX,2.0,lux
95,pek,NaN,NaN,NaN
101,pnq,PNQ,1.0,pnq


In [26]:
metro_node[metro_node.site_num.isna()].shape

(25, 4)

In [27]:
device_a = pd.DataFrame(nl_df.device_a.apply(split_site).to_list(), columns=['metro', 'device_num'])
device_z = pd.DataFrame(nl_df.device_z.apply(split_site).to_list(), columns=['metro', 'device_num'])

In [28]:
devices = pd.concat([device_a, device_z])
print(devices.shape)
devices.head(4)

(1533548, 2)


,metro,device_num
0,iad,55-br-fnc-f1-b2-t1-r9
1,lax,3-br-fnc-f1-b1-t1-r13
2,cmh,51-br-bfb-f1-b4-t1-r3
3,fra,53-br-fnc-f1-b1-t1-r5


In [29]:
uniq_devices = devices.drop_duplicates()
print(uniq_devices.shape)
uniq_devices.sample(4)

(3633, 2)


,metro,device_num
860,lhr,5-br-bfb-f1t1-r6
1620,ewr,53-br-bfb-f1-b8-t1-r4
401,pdx,1-br-fnc-f1-b1-t1-r12
7913,pdx,83-br-fnc-f1-b1-t1-r8


In [30]:
device_cnt = pd.DataFrame(uniq_devices.groupby('metro').count())
device_cnt = device_cnt.reset_index()
print(device_cnt.shape)
device_cnt.head(4)

(108, 2)


,metro,device_num
0,akl,3
1,ams,54
2,arn,27
3,ath,2


In [31]:
metro_node = metro_node[['nid', 'site_num']]

In [32]:
metro_node = pd.merge(metro_node, device_cnt, how='left', left_on='nid', right_on='metro')

In [33]:
print(metro_node.shape)
metro_node.head()

(133, 4)


,nid,site_num,metro,device_num
0,abc,NaN,NaN,NaN
1,akl,1.0,akl,3.0
2,amm,NaN,NaN,NaN
3,ams,5.0,ams,54.0
4,arn,4.0,arn,27.0


In [34]:
metro_node[metro_node.metro.isna()].shape

(25, 4)

In [35]:
metro_node = metro_node[['nid', 'site_num', 'device_num']]

In [36]:
metro_node.to_parquet('gs_data/metro.parquet', index=False)

In [37]:
port_node = pd.DataFrame({'nid': port_all})
print(port_node.shape)
port_node.sample(4)

(62334, 1)


,nid
37737,lhr54-br-fnc-f1-b1-t1-r9:jrp10-3
35660,kix56-br-fnc-f1-b1-t1-r5:jrp13-1
21621,iad140-br-fnc-f1-b1-t1-r6:jrp14-4
18292,iad117-br-fnc-f1-b1-t1-r4:jrp10-3


In [38]:
port_node.to_parquet('gs_data/port.parquet', index=False)

### Step 3: Generate edge tables

#### create all timestamps in the 2 hours

Traffic demands were collected in 5 minutes interval, starting from 0th minute of each hour. Althoug actual network traffic numbers were collected in 5 minutes interval too, however, starting in from every 4th minute of each hour.

Given demands should occur before actual traffic, we should align actual network traffic in the next round of time interval, i.e., shifting network traffic time 1 min later, e.g., 04:04:00 -> 04:05:00.

In [39]:
dm_ts_list = dm_df.timebucket.unique()
print(dm_ts_list)
print('-'.join(['-' for i in range(50)]))
nl_ts_list = nl_df.timestamp.unique()
print(nl_ts_list)

['2023-12-11T04:00:00.000Z' '2023-12-11T04:05:00.000Z'
 '2023-12-11T04:10:00.000Z' '2023-12-11T04:15:00.000Z'
 '2023-12-11T04:20:00.000Z' '2023-12-11T04:25:00.000Z'
 '2023-12-11T04:30:00.000Z' '2023-12-11T04:35:00.000Z'
 '2023-12-11T04:40:00.000Z' '2023-12-11T04:45:00.000Z'
 '2023-12-11T04:50:00.000Z' '2023-12-11T04:55:00.000Z'
 '2023-12-11T05:00:00.000Z' '2023-12-11T05:05:00.000Z'
 '2023-12-11T05:10:00.000Z' '2023-12-11T05:15:00.000Z'
 '2023-12-11T05:20:00.000Z' '2023-12-11T05:25:00.000Z'
 '2023-12-11T05:30:00.000Z' '2023-12-11T05:35:00.000Z'
 '2023-12-11T05:40:00.000Z' '2023-12-11T05:45:00.000Z'
 '2023-12-11T05:50:00.000Z' '2023-12-11T05:55:00.000Z']
---------------------------------------------------------------------------------------------------
['2023-12-11 04:04:00' '2023-12-11 04:09:00' '2023-12-11 04:14:00'
 '2023-12-11 04:19:00' '2023-12-11 04:24:00' '2023-12-11 04:29:00'
 '2023-12-11 04:34:00' '2023-12-11 04:39:00' '2023-12-11 04:44:00'
 '2023-12-11 04:49:00' '2023-12-11 04:

In [40]:
dm_df['ts'] = pd.to_datetime(dm_df.timebucket)
nl_df['ts'] = pd.to_datetime(nl_df.timestamp) + pd.Timedelta(minutes=1)

### Metro-demand-Metro real demands edge

In [41]:
dm_df = dm_df.sort_values(by=['src_region', 'dst_region', 'ts'])

In [42]:
m2m_gb_df = dm_df.groupby(['src_region', 'dst_region'])['gb'].apply(list).reset_index()
m2m_ts_df = dm_df.groupby(['src_region', 'dst_region'])['ts'].apply(list).reset_index()

In [43]:
print(m2m_gb_df.shape)
m2m_gb_df.head()

(10226, 3)


,src_region,dst_region,gb
0,akl,akl,"[7075.259733695001, 7260.10218911, 7279.018763..."
1,akl,ams,"[0.012662992, 0.011836268, 0.0124411879999999,..."
2,akl,arn,"[0.0612539959999999, 0.0672812879999999, 0.041..."
3,akl,ath,"[0.001068692, 0.001371152, 0.001048528, 0.0011..."
4,akl,atl,"[0.022039252, 0.0326051879999998, 0.0189138319..."


In [44]:
print(m2m_ts_df.shape)
m2m_ts_df.head()

(10226, 3)


,src_region,dst_region,ts
0,akl,akl,"[2023-12-11 04:00:00+00:00, 2023-12-11 04:05:0..."
1,akl,ams,"[2023-12-11 04:00:00+00:00, 2023-12-11 04:05:0..."
2,akl,arn,"[2023-12-11 04:00:00+00:00, 2023-12-11 04:05:0..."
3,akl,ath,"[2023-12-11 04:00:00+00:00, 2023-12-11 04:05:0..."
4,akl,atl,"[2023-12-11 04:00:00+00:00, 2023-12-11 04:05:0..."


In [45]:
m2m_df = pd.merge(m2m_gb_df, m2m_ts_df, how='inner')
print(m2m_df.shape)
m2m_df.head()

(10226, 4)


,src_region,dst_region,gb,ts
0,akl,akl,"[7075.259733695001, 7260.10218911, 7279.018763...","[2023-12-11 04:00:00+00:00, 2023-12-11 04:05:0..."
1,akl,ams,"[0.012662992, 0.011836268, 0.0124411879999999,...","[2023-12-11 04:00:00+00:00, 2023-12-11 04:05:0..."
2,akl,arn,"[0.0612539959999999, 0.0672812879999999, 0.041...","[2023-12-11 04:00:00+00:00, 2023-12-11 04:05:0..."
3,akl,ath,"[0.001068692, 0.001371152, 0.001048528, 0.0011...","[2023-12-11 04:00:00+00:00, 2023-12-11 04:05:0..."
4,akl,atl,"[0.022039252, 0.0326051879999998, 0.0189138319...","[2023-12-11 04:00:00+00:00, 2023-12-11 04:05:0..."


### Metro-to-Metro fully connected edges

In [46]:
metros = metro_node.nid

In [47]:
src_lists = []
dst_lists = []

for metro in metros:
    others = np.setdiff1d(metros, [metro])
    src_lists.append([metro]*len(others))
    dst_lists.append(others)

src_nodes = np.concatenate(src_lists)
dst_nodes = np.concatenate(dst_lists)

In [48]:
metro_edges = pd.DataFrame({'src_nid': src_nodes, 'dst_nid': dst_nodes})

In [49]:
metro_edges = pd.merge(metro_edges, m2m_df,
                       how='left',
                       left_on=['src_nid', 'dst_nid'],
                       right_on=['src_region', 'dst_region'])

In [50]:
print(metro_edges.shape)
metro_edges.sample(4)

(17556, 6)


,src_nid,dst_nid,src_region,dst_region,gb,ts
15695,tlv,tpe,tlv,tpe,"[0.046517876, 0.0499805, 0.132557228, 0.046478...","[2023-12-11 04:00:00+00:00, 2023-12-11 04:05:0..."
16111,vie,bah,vie,bah,"[1.5201224209999995, 1.3910861919999995, 1.474...","[2023-12-11 04:00:00+00:00, 2023-12-11 04:05:0..."
4386,dfw,cpt,dfw,cpt,"[0.133217656, 0.498407536, 0.066329358, 0.1489...","[2023-12-11 04:00:00+00:00, 2023-12-11 04:05:0..."
9748,lux,slc,NaN,NaN,NaN,NaN


In [51]:
one_ts = metro_edges.iloc[[3892]].ts
filled_val = list(one_ts)[0]
print(filled_val)

[Timestamp('2023-12-11 04:00:00+0000', tz='UTC'), Timestamp('2023-12-11 04:05:00+0000', tz='UTC'), Timestamp('2023-12-11 04:10:00+0000', tz='UTC'), Timestamp('2023-12-11 04:15:00+0000', tz='UTC'), Timestamp('2023-12-11 04:20:00+0000', tz='UTC'), Timestamp('2023-12-11 04:25:00+0000', tz='UTC'), Timestamp('2023-12-11 04:30:00+0000', tz='UTC'), Timestamp('2023-12-11 04:35:00+0000', tz='UTC'), Timestamp('2023-12-11 04:40:00+0000', tz='UTC'), Timestamp('2023-12-11 04:45:00+0000', tz='UTC'), Timestamp('2023-12-11 04:50:00+0000', tz='UTC'), Timestamp('2023-12-11 04:55:00+0000', tz='UTC'), Timestamp('2023-12-11 05:00:00+0000', tz='UTC'), Timestamp('2023-12-11 05:05:00+0000', tz='UTC'), Timestamp('2023-12-11 05:10:00+0000', tz='UTC'), Timestamp('2023-12-11 05:15:00+0000', tz='UTC'), Timestamp('2023-12-11 05:20:00+0000', tz='UTC'), Timestamp('2023-12-11 05:25:00+0000', tz='UTC'), Timestamp('2023-12-11 05:30:00+0000', tz='UTC'), Timestamp('2023-12-11 05:35:00+0000', tz='UTC'), Timestamp('2023-12-

In [52]:
metro_ts = metro_edges.ts.apply(lambda x: x if isinstance(x, list) else filled_val)
metro_gb = metro_edges.gb.apply(lambda x: x if isinstance(x, list) else [0] * 24)

In [53]:
mdm_edges = metro_edges[['src_nid', 'dst_nid']]
mdm_edges['ts'] = metro_ts
mdm_edges['gb'] = metro_gb

/tmp/ipykernel_14659/26004180.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mdm_edges['ts'] = metro_ts
/tmp/ipykernel_14659/26004180.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mdm_edges['gb'] = metro_gb


In [54]:
mdm_edges.sample(7)

,src_nid,dst_nid,ts,gb
3025,cbr,vie,"[2023-12-11 04:00:00+00:00, 2023-12-11 04:05:0...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
14703,sin,hio,"[2023-12-11 04:00:00+00:00, 2023-12-11 04:05:0...","[57.058931892, 56.656763906, 54.13195374, 54.6..."
4260,den,dus,"[2023-12-11 04:00:00+00:00, 2023-12-11 04:05:0...","[0.981139912, 0.013394112, 0.005323296, 0.0129..."
3144,ccu,sfo,"[2023-12-11 04:00:00+00:00, 2023-12-11 04:05:0...","[0.0247597119999999, 0.0200833439999998, 0.012..."
11466,mxp,syd,"[2023-12-11 04:00:00+00:00, 2023-12-11 04:05:0...","[10.590685302, 8.624741951999999, 12.78750033,..."
10783,mia,otp,"[2023-12-11 04:00:00+00:00, 2023-12-11 04:05:0...","[0.009517408, 0.007299368, 0.0103844599999999,..."
13858,qro,zhy,"[2023-12-11 04:00:00+00:00, 2023-12-11 04:05:0...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [55]:
mdm_edges.to_parquet('gs_data/metro-demand-metro.parquet', index=False)

### Port-send-Port edges

In [56]:
p2p_edges = nl_df[['da_pa', 'dz_pz', 'ts',
                   'lifecycle_state', 'tshift_status_a', 'tshift_status_z',
                   'fabric', 'span_type', 'span_build_type',
                   'traffic_gbps']]

In [102]:
p2p_edges[(p2p_edges.da_pa=='maa50-br-fnc-f1-b1-t1-r15:jrp6-2') & (p2p_edges.dz_pz=='maa51-br-fnc-f1-b1-t1-r15:jrp6-2')]

,da_pa,dz_pz,ts,lifecycle_state,tshift_status_a,tshift_status_z,fabric,span_type,span_build_type,traffic_gbps
210483,maa50-br-fnc-f1-b1-t1-r15:jrp6-2,maa51-br-fnc-f1-b1-t1-r15:jrp6-2,2023-12-11 05:35:00,OPERATIONAL,not_shifted,not_shifted,FNC,INTRA-METRO,AMAZON_OLP_METRO,3.283874
242431,maa50-br-fnc-f1-b1-t1-r15:jrp6-2,maa51-br-fnc-f1-b1-t1-r15:jrp6-2,2023-12-11 05:40:00,OPERATIONAL,not_shifted,not_shifted,FNC,INTRA-METRO,AMAZON_OLP_METRO,3.454522
274379,maa50-br-fnc-f1-b1-t1-r15:jrp6-2,maa51-br-fnc-f1-b1-t1-r15:jrp6-2,2023-12-11 05:45:00,OPERATIONAL,not_shifted,not_shifted,FNC,INTRA-METRO,AMAZON_OLP_METRO,3.280173
306327,maa50-br-fnc-f1-b1-t1-r15:jrp6-2,maa51-br-fnc-f1-b1-t1-r15:jrp6-2,2023-12-11 05:50:00,OPERATIONAL,not_shifted,not_shifted,FNC,INTRA-METRO,AMAZON_OLP_METRO,3.404013
338275,maa50-br-fnc-f1-b1-t1-r15:jrp6-2,maa51-br-fnc-f1-b1-t1-r15:jrp6-2,2023-12-11 05:55:00,OPERATIONAL,not_shifted,not_shifted,FNC,INTRA-METRO,AMAZON_OLP_METRO,3.502527
370223,maa50-br-fnc-f1-b1-t1-r15:jrp6-2,maa51-br-fnc-f1-b1-t1-r15:jrp6-2,2023-12-11 06:00:00,OPERATIONAL,not_shifted,not_shifted,FNC,INTRA-METRO,AMAZON_OLP_METRO,3.362675


In [58]:
p2p_fabric = p2p_edges.groupby(['da_pa', 'dz_pz'])['fabric'].apply(set).reset_index()
p2p_fabric['num_type'] = p2p_fabric.fabric.apply(lambda x: len(x))
print(p2p_fabric.shape)

(31373, 4)


In [59]:
p2p_fabric[p2p_fabric.num_type > 1]

,da_pa,dz_pz,fabric,num_type


In [60]:
p2p_span = p2p_edges.groupby(['da_pa', 'dz_pz'])['span_type'].apply(set).reset_index()
p2p_span['num_type'] = p2p_span.span_type.apply(lambda x: len(x))
print(p2p_span.shape)
p2p_span[p2p_span.num_type > 1]

(31373, 4)


,da_pa,dz_pz,span_type,num_type


In [61]:
p2p_span_bt = p2p_edges.groupby(['da_pa', 'dz_pz'])['span_build_type'].apply(set).reset_index()
p2p_span_bt['num_type'] = p2p_span_bt.span_build_type.apply(lambda x: len(x))
print(p2p_span_bt.shape)
p2p_span_bt[p2p_span_bt.num_type > 1]

(31373, 4)


,da_pa,dz_pz,span_build_type,num_type
2227,bom50-br-fnc-f1-b1-t1-r13:jrp3-1,bom78-br-fnc-f1-b1-t1-r13:jrp3-1,"{AMAZON_OLP_METRO, LEASED}",2
2228,bom50-br-fnc-f1-b1-t1-r13:jrp3-2,bom78-br-fnc-f1-b1-t1-r13:jrp3-2,"{AMAZON_OLP_METRO, LEASED}",2
2229,bom50-br-fnc-f1-b1-t1-r13:jrp3-3,bom78-br-fnc-f1-b1-t1-r13:jrp3-3,"{AMAZON_OLP_METRO, LEASED}",2
2230,bom50-br-fnc-f1-b1-t1-r13:jrp3-4,bom78-br-fnc-f1-b1-t1-r13:jrp3-4,"{AMAZON_OLP_METRO, LEASED}",2
2231,bom50-br-fnc-f1-b1-t1-r13:jrp4-1,bom78-br-fnc-f1-b1-t1-r13:jrp4-1,"{AMAZON_OLP_METRO, LEASED}",2
...,...,...,...,...
21346,mrs51-br-bfb-f1-b1-t1-r4:et-0/0/35,mrs52-br-bfb-f1-b2-t1-r4:et-0/0/1,"{AMAZON_OLP_METRO, LEASED}",2
21348,mrs51-br-bfb-f1-b1-t1-r5:et-0/0/17,mrs52-br-bfb-f1-b2-t1-r5:et-0/0/1,"{AMAZON_OLP_METRO, LEASED}",2
21349,mrs51-br-bfb-f1-b1-t1-r5:et-0/0/35,mrs52-br-bfb-f1-b2-t1-r5:et-0/0/5,"{AMAZON_OLP_METRO, LEASED}",2
21351,mrs51-br-bfb-f1-b1-t1-r6:et-0/0/17,mrs52-br-bfb-f1-b2-t1-r6:et-0/0/1,"{AMAZON_OLP_METRO, LEASED}",2


In [62]:
p2p_edges = p2p_edges.sort_values(by=['da_pa', 'dz_pz', 'ts'])

In [63]:
p2p_ts = p2p_edges.groupby(['da_pa', 'dz_pz'])['ts'].apply(list).reset_index()
p2p_tr = p2p_edges.groupby(['da_pa', 'dz_pz'])['traffic_gbps'].apply(list).reset_index()

In [64]:
p2p_tr.iloc[3502]

da_pa                              cdg52-br-bfb-f1t1-r11:et-0/0/5
dz_pz                          lhr61-br-bfb-f1-b7-t1-r6:et-0/0/11
traffic_gbps    [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
Name: 3502, dtype: object

In [103]:
p2p_tr['l'] = p2p_tr.traffic_gbps.apply(len)
print(p2p_tr.l.unique())
p2p_irregular = p2p_tr[p2p_tr.l!=24]
print(p2p_irregular.shape)

[24 48 15  6 16]
(608, 5)


In [105]:
p2p_irregular[['da_pa', 'dz_pz', 'traffic_gbps', 'l']].to_csv('irregular_port_pairs.csv', index=False)

In [66]:
def correct_traffic_list(traffic_list):
    length = len(traffic_list)

    if length < 24:                                                     # pad 0s to short list
        new_traffic_list = traffic_list + [0] * (24 - length)
    elif length == 24:                                                  # do nothing on correct length ones
        new_traffic_list = traffic_list
    elif length > 24 and length != 48:                                  # trunk the first 24 values for non-48 long list
        new_traffic_list = traffic_list[:24]
    else:                                                               # take the odd number from a 48 long list
        new_traffic_list = [traffic_list[i] for i in range(0, 48, 2)]

    return new_traffic_list

In [67]:
p2p_tr['new_traffic_gbps'] = p2p_tr.traffic_gbps.apply(correct_traffic_list)
p2p_tr['l'] = p2p_tr.new_traffic_gbps.apply(len)
print(p2p_tr.l.unique())

[24]


In [68]:
p2p_ts['l'] = p2p_ts.ts.apply(len)
print(p2p_ts.l.unique())

[24 48 15  6 16]


In [69]:
def apply_event(row):
    """
    """
    event = []
    for lf, st_a, st_z in zip(row['lifecycle_state'], row['tshift_status_a'], row['tshift_status_z']):
        if (lf != "OPERATIONAL") or (st_a == "shifted" or (st_z == "shifted")):
            event.append(0)
        else:
            event.append(1)
    return pd.Series({'event': event})

p2p_event = p2p_edges.groupby(['da_pa', 'dz_pz']).apply(apply_event).reset_index()

In [93]:
def correct_event_list(event_list):
    length = len(event_list)

    if length < 24:                                                     # pad 1s to short list
        new_event_list = event_list + [1] * (24 - length)
    elif length == 24:                                                  # do nothing on correct length ones
        new_event_list = event_list
    elif length > 24 and length != 48:                                  # trunk the first 24 values for non-48 long list
        new_event_list = event_list[:24]
    else:                                                               # take the odd number from a 48 long list
        new_event_list = [event_list[i] for i in range(0, 48, 2)]

    return new_event_list

In [94]:
p2p_event['new_event'] = p2p_event.event.apply(correct_event_list)
p2p_event['l'] = p2p_event.new_event.apply(len)
print(p2p_event.l.unique())

[24]


In [71]:
p2p_edges.head(4)

,da_pa,dz_pz,ts,lifecycle_state,tshift_status_a,tshift_status_z,fabric,span_type,span_build_type,traffic_gbps
6877,akl50-br-cor-r1:et-0/0/19,syd1-br-cor-r2:et-5/1/0,2023-12-11 04:05:00,OPERATIONAL,not_shifted,not_shifted,CLASSIC,BACKBONE,BACKBONE,7.866495
38828,akl50-br-cor-r1:et-0/0/19,syd1-br-cor-r2:et-5/1/0,2023-12-11 04:10:00,OPERATIONAL,not_shifted,not_shifted,CLASSIC,BACKBONE,BACKBONE,7.762273
70779,akl50-br-cor-r1:et-0/0/19,syd1-br-cor-r2:et-5/1/0,2023-12-11 04:15:00,OPERATIONAL,not_shifted,not_shifted,CLASSIC,BACKBONE,BACKBONE,8.038799
102730,akl50-br-cor-r1:et-0/0/19,syd1-br-cor-r2:et-5/1/0,2023-12-11 04:20:00,OPERATIONAL,not_shifted,not_shifted,CLASSIC,BACKBONE,BACKBONE,8.110079


#### Merge all p2p feature dataframes

- p2p_fabric; current data shows there is only ONE fabric type for each edge.
- p2p_span; current data shows there is only ONE span type for each edge.
- p2p_span_bt;current data has multiple span build types for one edge.
- p2p_ts;  varient length, so will not used in data set
- p2p_tr;
- p2p_event.

In [72]:
p2p_fabric['fabric_str'] = p2p_fabric['fabric'].apply(lambda x: x.pop() if len(x) > 0 else '')
p2p_fabric.head(4)

,da_pa,dz_pz,fabric,num_type,fabric_str
0,akl50-br-cor-r1:et-0/0/19,syd1-br-cor-r2:et-5/1/0,{},1,CLASSIC
1,akl50-br-cor-r2:et-0/0/19,syd1-br-cor-r3:et-1/3/0,{},1,CLASSIC
2,akl50-br-cor-r3:et-0/0/19,syd62-br-bfb-f1-b2-t1-r2:et-0/0/13,{},1,CLASSIC
3,ams1-br-bfb-f1-b2-t1-r1:et-0/0/13,fra60-br-bfb-f1-b4-t1-r3:et-0/0/7,{},1,CLASSIC


In [73]:
p2p_f = p2p_fabric[['da_pa', 'dz_pz', 'fabric_str']]
print(p2p_f.shape)

(31373, 3)


In [74]:
p2p_span['span_str'] = p2p_span['span_type'].apply(lambda x: x.pop() if len(x) > 0 else '')
p2p_span.sample(4)

,da_pa,dz_pz,span_type,num_type,span_str
26433,pdx2-br-fnc-f1-b3-t1-r2:jrp1-3,pdx69-br-fnc-f1-b1-t1-r14:jrp1-3,{},1,INTRA-METRO
25444,pdx1-br-fnc-f1-b3-t1-r3:jrp8-1,pdx68-br-fnc-f1-b1-t1-r15:jrp12-1,{},1,INTRA-METRO
12290,iad12-br-fnc-f1-b2-t1-r6:jrp16-3,iad79-br-fnc-f1-b1-t1-r6:jrp8-3,{},1,INTRA-METRO
543,ams54-br-fnc-f1-b1-t1-r16:jrp6-3,ams58-br-fnc-f1-b1-t1-r16:jrp6-3,{},1,INTRA-METRO


In [75]:
p2p_s = p2p_span[['da_pa', 'dz_pz', 'span_str']]
print(p2p_s.shape)

(31373, 3)


In [76]:
def process_span_build_type(bt_set):
    if len(bt_set) == 0:
        return ''
    elif len(bt_set) == 1:
        return bt_set.pop()
    else:
        return ':'.join([bt for bt in bt_set])

p2p_span_bt['span_bt_str'] = p2p_span_bt['span_build_type'].apply(process_span_build_type)
p2p_span_bt.sample(4)

,da_pa,dz_pz,span_build_type,num_type,span_bt_str
14325,iad55-br-fnc-f1-b2-t1-r9:jrp11-1,iad61-br-fnc-f1-b1-t1-r1:jrp15-1,{},1,AMAZON_OLP_METRO
4783,cmh59-br-fnc-f1-b2-t1-r5:jrp5-1,cmh61-br-fnc-f1-b2-t1-r5:jrp1-1,{},1,AMAZON_OLP_METRO
468,ams54-br-bfb-f1-b4-t1-r5:et-0/0/1,lhr61-br-bfb-f1-b8-t1-r3:et-0/0/5,{},1,BACKBONE
24433,pdx1-br-fnc-f1-b1-t1-r13:jrp8-4,pdx80-br-fnc-f1-b1-t1-r13:jrp8-4,{},1,AMAZON_OLP_METRO


In [77]:
p2p_sbt = p2p_span_bt[['da_pa', 'dz_pz', 'span_bt_str']]
print(p2p_sbt.shape)

(31373, 3)


In [78]:
print(p2p_ts.shape)
print(p2p_tr.shape)
print(p2p_event.shape)

(31373, 4)
(31373, 5)
(31373, 3)


In [97]:
p2p = pd.merge(p2p_f, p2p_s, left_on=['da_pa', 'dz_pz'], right_on=['da_pa', 'dz_pz'])
p2p = pd.merge(p2p, p2p_sbt, left_on=['da_pa', 'dz_pz'], right_on=['da_pa', 'dz_pz'])
# p2p = pd.merge(p2p, p2p_ts, left_on=['da_pa', 'dz_pz'], right_on=['da_pa', 'dz_pz'])
p2p = pd.merge(p2p, p2p_tr[['da_pa', 'dz_pz', 'new_traffic_gbps']], left_on=['da_pa', 'dz_pz'], right_on=['da_pa', 'dz_pz'])
p2p = pd.merge(p2p, p2p_event[['da_pa', 'dz_pz', 'new_event']], left_on=['da_pa', 'dz_pz'], right_on=['da_pa', 'dz_pz'])

In [98]:
p2p.sample(7)

,da_pa,dz_pz,fabric_str,span_str,span_bt_str,new_traffic_gbps,new_event
16153,iad6-br-fnc-f1-b3-t1-r10:jrp4-1,iad53-br-fnc-f1-b1-t1-r10:jrp4-1,FNC,INTRA-METRO,AMAZON_OLP_METRO,"[24.0024986, 23.5958211, 22.3387505, 22.159689...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
25124,pdx1-br-fnc-f1-b3-t1-r13:jrp16-1,pdx62-br-fnc-f1-b1-t1-r5:jrp16-1,FNC,INTRA-METRO,BULK_FIBER,"[6.1957335, 6.4918953, 6.2354744, 6.8462449, 6...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
19210,kix50-br-fnc-f1-b1-t1-r15:jrp10-1,nrt51-br-fnc-f1-b1-t1-r15:jrp10-1,FNC,INTRA-METRO,AMAZON_OLP_METRO,"[8.6072586, 8.1212708, 8.759042, 8.5488328, 8....","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
3312,cdg50-br-fnc-f1-b1-t1-r15:jrp12-1,cdg55-br-fnc-f1-b1-t1-r15:jrp12-1,FNC,INTRA-METRO,AMAZON_OLP_METRO,"[4.3021170000000005e-06, 4.2867886e-06, 4.3000...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
25034,pdx1-br-fnc-f1-b2-t1-r1:jrp2,pdx4-br-fnc-f1-b1-t1-r9:jrp7,FNC,INTRA-METRO,AMAZON_OLP_METRO,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
8951,fra53-br-fnc-f1-b2-t1-r7:jrp5-4,fra90-br-fnc-f1-b2-t1-r15:jrp5-4,FNC,INTRA-METRO,AMAZON_OLP_METRO,"[3.4519760000000005e-05, 3.8200836e-05, 4.3392...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
27381,pdx56-br-fnc-f1-b1-t1-r5:jrp14-1,pdx66-br-fnc-f1-b1-t1-r1:jrp6-1,FNC,INTRA-METRO,BULK_FIBER,"[3.27158784, 2.98065178, 3.07278259, 2.8992560...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."


In [99]:
p2p.to_parquet('gs_data/port-to-port.parquet', index=False)

### Port belong-to Metro edges

Extract the first 3 letters from each Port, and match them back to the Metro table. Should have no missing. Otherwise, add it to the Metro table.
Based the splited Port string, create the Port to Metro edges.

In [82]:
port_node.sample(7)

,nid
11461,dub58-br-fnc-f1-b1-t1-r10:jrp5-2
6179,cmh51-br-fnc-f1-b1-t1-r13:jrp2-1
34146,jfk1-br-fnc-f1-b1-t1-r16:jrp5-4
37764,lhr54-br-fnc-f1-b1-t1-r9:jrp6-2
3212,bom50-br-fnc-f1-b1-t1-r16:jrp1-4
14138,fra56-br-bfb-f1-b2-t1-r1:et-0/0/13
14403,fra56-br-fnc-f1-b2-t1-r2:jrp8-3


In [83]:
port_metro = port_node.nid.apply(lambda x: x[:3] if x is not None else '')

In [84]:
port_metro

0        akl
1        akl
2        akl
3        ams
4        ams
        ... 
62329    zrh
62330    zrh
62331    zrh
62332    zrh
62333    zrh
Name: nid, Length: 62334, dtype: object

In [85]:
port_metro_unique = pd.DataFrame({'metro_id':port_metro.unique()})
port_metro_unique.shape

(108, 1)

In [86]:
port_metro_unique.sample(4)

,metro_id
107,zrh
19,cbr
0,akl
57,lis


In [87]:
diff = pd.merge(port_metro_unique, metro_node, left_on='metro_id', right_on='nid', how='inner')
diff.shape

(108, 4)

It confirms that all metros extracted from port strings are included in the Metro table.

In [88]:
port_metro_edge = pd.DataFrame({'pid':port_node.nid, 'mid': port_metro})
print(port_metro_edge.shape)
port_metro_edge.head(4)

(62334, 2)


,pid,mid
0,akl50-br-cor-r1:et-0/0/19,akl
1,akl50-br-cor-r2:et-0/0/19,akl
2,akl50-br-cor-r3:et-0/0/19,akl
3,ams1-br-bfb-f1-b2-t1-r1:et-0/0/13,ams


In [89]:
metro_port_edge = pd.DataFrame({'mid': port_metro, 'pid':port_node.nid})

In [90]:
port_metro_edge.to_parquet('gs_data/port-in-metro.parquet', index=False)
metro_port_edge.to_parquet('gs_data/metro-in-rev-port.parquet', index=False)